In [1]:
import glob
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import csv

pd.options.display.float_format = '{:.5f}'.format

In [2]:
use_counties = ['台北市', '高雄市', '新北市', '桃園市', '台中市', '台南市', '苗栗縣', '新竹縣', '基隆市',
       '屏東縣', '新竹市', '宜蘭縣', '花蓮縣', '嘉義市', '金門縣', '嘉義縣', '彰化縣', '雲林縣']
dfs = []
for pathto_file in glob.glob('../外部資料集/實價登錄/*/*_a.csv'):
    df = pd.read_csv(pathto_file, on_bad_lines='skip', skiprows=[1])
    df['單價元平方公尺'].fillna(0)
    df['土地位置建物門牌'] = df['土地位置建物門牌'].apply(lambda x: (x.replace('臺北市', '台北市').replace('臺中市', '台中市').replace('臺南市', '台南市')))
    county = df['土地位置建物門牌'].apply(lambda x: (x+'123')[:3]).value_counts().index[0]
    if county not in use_counties:
        continue
    df['縣市'] = county
    avg_price = df.query('單價元平方公尺 > 1000')['單價元平方公尺'].median()
    avg_price = avg_price*0.1
    df = df.query(f'單價元平方公尺 > {avg_price}')
    df['source'] = pathto_file.split('/')[-2]
    dfs.append(df)
df = pd.concat(dfs).reset_index(drop=True)

/tmp/ipykernel_20965/16003613.py:5: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(pathto_file, on_bad_lines='skip', skiprows=[1])
/tmp/ipykernel_20965/16003613.py:5: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(pathto_file, on_bad_lines='skip', skiprows=[1])


In [3]:
set(use_counties) - set(df['縣市'].unique().tolist())

set()

In [4]:
df['縣市'].value_counts()

縣市
新北市    169523
台中市    152859
桃園市    143686
高雄市    123617
台南市     91415
台北市     69240
彰化縣     37424
新竹縣     36707
屏東縣     30367
新竹市     24135
苗栗縣     22713
宜蘭縣     22350
雲林縣     19550
基隆市     15708
嘉義縣     15128
花蓮縣     11842
嘉義市     11150
金門縣      2828
Name: count, dtype: int64

In [5]:
df.groupby('鄉鎮市區')['單價元平方公尺'].mean()

鄉鎮市區
七堵區    57159.55094
七股區    29483.78882
三地門鄉   11704.94444
三峽區    81981.61727
三星鄉    43748.12832
           ...    
鼓山區    81831.20558
龍井區    52092.59899
龍崎區    17213.25000
龍潭區    49422.33740
龜山區    77609.76650
Name: 單價元平方公尺, Length: 318, dtype: float64

In [6]:
df = df.query('主建物面積 >= 2').copy().reset_index(drop=True)
df = df.query('主建物面積 <= 2000').copy().reset_index(drop=True)
df = df.query('陽台面積 <= 100').copy().reset_index(drop=True)
df = df.query('單價元平方公尺 >= 1000')
df = df.query('單價元平方公尺 <= 2000000').reset_index(drop=True)

col = '主建物面積'
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '陽台面積'
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '附屬建物面積'
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '車位面積'
df[col] = df['車位移轉總面積平方公尺'].values
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '建物面積'
df[col] = df['建物移轉總面積平方公尺'].values
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '土地面積'
df[col] = df['土地移轉總面積平方公尺'].values
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

# col = '單價'
# df[col] = df['單價元平方公尺'].values
# scaler = StandardScaler()
# scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
# df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1)) + 2
# print(df[col].describe())

count   804226.00000
mean        -0.00000
std          1.00000
min         -2.57025
25%         -0.62143
50%         -0.19504
75%          0.41113
max         11.23237
Name: 主建物面積, dtype: float64
count   804226.00000
mean         0.00000
std          1.00000
min         -1.50560
25%         -0.50668
50%          0.16669
75%          0.64213
max          5.10059
Name: 陽台面積, dtype: float64
count   804226.00000
mean         0.00000
std          1.00000
min         -0.65162
25%         -0.65162
50%         -0.65162
75%          0.58061
max         18.59469
Name: 附屬建物面積, dtype: float64
count   804226.00000
mean        -0.00000
std          1.00000
min         -0.70228
25%         -0.70228
50%         -0.70228
75%          1.09216
max         15.21460
Name: 車位面積, dtype: float64
count   804226.00000
mean        -0.00000
std          1.00000
min         -3.54815
25%         -0.60027
50%         -0.06949
75%          0.47735
max         46.11822
Name: 建物面積, dtype: float64
count   804226.00000
m

In [7]:
def get_year_from_str(x):
    try:
        x = int(x)
        if x < 1000101:
            return int(str(x)[0:2])
        else:
            return int(str(x)[0:3])
    except:
        return -1

def get_month_from_str(x):
    try:
        x = int(x)
        if x < 1000101:
            return int(str(x)[2:4])
        else:
            return int(str(x)[3:5])
    except:
        return -1

In [8]:
df['Trade_Year'] = df['交易年月日'].apply(get_year_from_str).astype(int) + 1911
df['Trade_Month'] = df['交易年月日'].apply(get_month_from_str).astype(int)
df['Trade_YearMonth'] = df['Trade_Year']*100 + df['Trade_Month']

In [9]:
df = df[~df['建築完成年月'].isna()].copy().reset_index(drop=True)

In [10]:
df['Build_Year'] = df['建築完成年月'].apply(get_year_from_str).astype(int) + 1911
df['Build_Month'] = df['建築完成年月'].apply(get_month_from_str).astype(int)

In [11]:
df = df.query('Build_Year > 1910')
df = df.query('Build_Month > 0 and Build_Month <= 12').copy().reset_index(drop=True)

In [12]:
pd.set_option('display.max_columns', None)
df = df.query('Trade_Year >= 2019 and Trade_Year <= 2023')
df = df.query('Trade_Month >= 1 and Trade_Month <= 12')

In [13]:
df['屋齡'] = df['Trade_Year'] - df['Build_Year'] + (df['Trade_Month'] - df['Build_Month'])/12
df = df.query('屋齡 >= 0 and 屋齡 <= 60.1').copy().reset_index(drop=True)
df['屋齡'].describe()

count   600078.00000
mean        20.62706
std         15.41682
min          0.00000
25%          5.16667
50%         22.58333
75%         31.75000
max         60.08333
Name: 屋齡, dtype: float64

In [14]:
df['交易標的'].value_counts()

交易標的
房地(土地+建物)       373759
房地(土地+建物)+車位    223215
建物                3104
Name: count, dtype: int64

In [15]:
df['建物型態'].value_counts()

建物型態
住宅大樓(11層含以上有電梯)    242278
透天厝                153484
華廈(10層含以下有電梯)      108417
公寓(5樓含以下無電梯)        80984
套房(1房1廳1衛)           9565
店面(店鋪)               2562
辦公商業大樓               1446
廠辦                    694
其他                    648
Name: count, dtype: int64

In [16]:
df = df[~df['總樓層數'].isna()].copy()
df = df[~df['總樓層數'].isna()].copy()
df = df.query('交易標的 != "土地"').copy()
df = df.reset_index(drop=True)
df['交易標的'].value_counts()

交易標的
房地(土地+建物)       373257
房地(土地+建物)+車位    223162
建物                3100
Name: count, dtype: int64

In [17]:
df['單價元平方公尺'].apply(lambda x: x / 60000) .describe()

count   599519.00000
mean         1.44753
std          1.04816
min          0.03172
25%          0.82767
50%          1.17173
75%          1.73728
max         32.80000
Name: 單價元平方公尺, dtype: float64

In [18]:
floor_dict = {
    '二十': 20,
    '三十': 30,
    '四十': 40,
    '五十': 50,
    '六十': 60,
    '七十': 70,
    '八十': 80,
    '九十': 90,
}
floor_dict2 = {
    '一': 1,
    '二': 2,
    '三': 3,
    '四': 4,
    '五': 5,
    '六': 6,
    '七': 7,
    '八': 8,
    '九': 9,
}
def get_floor(x):
    x = str(x)
    floor = 0
    for k, v in floor_dict.items():
        if x.startswith(k):
            x = x.replace(k, '')
            floor += v
    if x.startswith('十'):
        x = x.replace('十', '')
        floor += 10
    for k, v in floor_dict2.items():
        if x.startswith(k):
            x = x.replace(k, '')
            floor += v
    return floor

df['total_floor'] = df['總樓層數'].apply(get_floor)
df['floor'] = df['移轉層次'].apply(get_floor)

In [19]:
def get_road_name(row):
    road_name = row['土地位置建物門牌']
    if road_name.startswith(row['縣市']):
        road_name = road_name.replace(row['縣市'], '')
    if road_name.startswith(row['鄉鎮市區']):
        road_name = road_name.replace(row['鄉鎮市區'], '')
    if '段' in road_name:
        road_name = road_name.split('段')[0] + '段'
    elif '路' in road_name:
        road_name = road_name.split('路')[0] + '路'
    elif '街' in road_name:
        road_name = road_name.split('街')[0] + '街'
    else:
        road_name = ""
    return road_name
    
df['路名'] = df.apply(lambda row: get_road_name(row), axis=1)

In [20]:
df = df.query('floor > 0').reset_index(drop=True)
df = df.query('total_floor > 0').reset_index(drop=True)
df = df.query('total_floor >= floor').reset_index(drop=True)
df = df.query('路名 != ""').reset_index(drop=True)

In [21]:
df.sample(5)

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,縣市,source,車位面積,建物面積,土地面積,Trade_Year,Trade_Month,Trade_YearMonth,Build_Year,Build_Month,屋齡,total_floor,floor,路名
36923,北投區,房地(土地+建物)+車位,台北市北投區中央北路四段１８巷４６弄１８號５樓,48.66000,住,NaN,NaN,1090403,土地1建物1車位1,五層,五層,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,1001206.00000,144.91000,2,2,2,有,有,15000000,103513.00000,坡道平面,0.00000,0,含增建或未登記建物。；,RPROMLPJNHPFFEA38CA,-0.54636,1.75961,-0.12425,NaN,NaN,台北市,2020Q2,-0.70228,0.11684,0.21018,2020,4,202004,2011,12,8.33333,5,5,中央北路四段
432165,中壢區,房地(土地+建物),桃園市中壢區龍川五街９號二樓,20.05000,住,NaN,NaN,1120726,土地1建物1車位0,二層,七層,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,830926.00000,112.01000,3,2,2,有,有,6300000,56245.00000,NaN,0.00000,0,其他增建:陽台加裝鐵窗,RPTPMLSLQHIGFBH76DA,-0.16321,0.55800,0.64824,有,1195.00000,桃園市,2023Q3,-0.70228,-0.32595,-0.35195,2023,7,202307,1994,9,28.83333,7,2,龍川五街
346611,吉安鄉,房地(土地+建物),花蓮縣吉安鄉全民街９５號五樓,42.48000,NaN,鄉村區,乙種建築用地,1110730,土地1建物1車位0,五層,五層,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,880414.00000,112.64000,3,2,2,有,無,3600000,31960.00000,NaN,0.00000,0,NaN,RPOOMLKKQHHGFAU98DA,-0.12137,0.91234,0.40030,有,321.00000,花蓮縣,2022Q4,-0.70228,-0.31690,0.10712,2022,7,202207,1999,4,23.25000,5,5,全民街
315979,花蓮市,房地(土地+建物),花蓮縣花蓮市國盛二街２１３號四樓之九,4.64000,商,NaN,NaN,1110712,土地1建物1車位0,四層,九層,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,0821105,39.60000,1,0,1,有,有,1850000,46717.00000,NaN,0.00000,0,NaN,RPPOMLMLPHHGFAU66DA,-1.36691,-0.65162,-1.50560,有,NaN,花蓮縣,2022Q3,-0.70228,-1.63226,-0.87486,2022,7,202207,1993,11,28.66667,9,4,國盛二街
324763,中正區,房地(土地+建物),基隆市中正區新豐街２９６號１６樓,10.42000,住,NaN,NaN,1110922,土地1建物1車位0,十六層,二十三層,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,851129.00000,53.75000,2,1,1,有,有,3530000,65674.00000,NaN,0.00000,0,NaN,RPOOMLSLRHHGFDC48DA,-1.08073,-0.65162,0.05430,有,297.00000,基隆市,2022Q4,-0.70228,-1.31605,-0.63319,2022,9,202209,1996,11,25.83333,23,16,新豐街


In [22]:
df['總樓層數'] = df['total_floor']
df['移轉層次'] = df['floor']

In [23]:
def get_car_cnt(x):
    x = x.split('車位')[-1]
    if x.isdigit():
        return int(x)
    return -999999

In [24]:
df['車位個數'] = df['交易筆棟數'].apply(get_car_cnt).astype(int)
df['車位個數'].describe()

count   439291.00000
mean         0.58584
std          0.75884
min          0.00000
25%          0.00000
50%          0.00000
75%          1.00000
max         79.00000
Name: 車位個數, dtype: float64

In [25]:
df['單價'] = df['單價元平方公尺'].apply(lambda x: x/58000)
df['單價'].describe()

count   439291.00000
mean         1.60893
std          1.07374
min          0.04257
25%          0.91404
50%          1.30391
75%          1.97874
max         31.96309
Name: 單價, dtype: float64

In [26]:
df = df.query('單價 > 0').copy().reset_index(drop=True)

In [27]:
df['格局_房'] = df['建物現況格局-房']
df['格局_廳'] = df['建物現況格局-廳']
df['格局_衛'] = df['建物現況格局-衛']
df['格局_隔間'] = df['建物現況格局-隔間'].map({'有': True, '無': False})
#df['格局_電梯'] = df['電梯'].map({'有': True, '無': False})
df['格局_管理'] = df['有無管理組織'].map({'有': True, '無': False})

In [28]:
dfs_build = []
for pathto_file in glob.glob('../外部資料集/實價登錄/*/*_a_build.csv'):
    _df = pd.read_csv(pathto_file, skiprows=[1])
    dfs_build.append(_df)
df_build = pd.concat(dfs_build).reset_index(drop=True)

useages = ['住家用', '集合住宅', '其他', '店鋪', '商業用', '國民住宅', '住工用', '一般事務所', '住商用',
       '廠房', '工業用', '辦公室']
print(df_build.shape)
print(set(useages) - set(df_build.主要用途.unique().tolist()))
df_build = df_build[df_build['主要用途'].isin(useages)]
print(df_build.shape)
df_build = df_build.drop_duplicates(subset=['編號'], keep='first')
print(df_build.shape)

df_build_usage_map = dict(zip(df_build['編號'].values, df_build['主要用途'].values))

df['主要用途_from_info'] = df['編號'].map(df_build_usage_map)
df = df[~df['主要用途_from_info'].isna()].copy().reset_index(drop=True)
df = df[~df['主要用途'].isna()].copy().reset_index(drop=True)
print(df['主要用途'].value_counts())
print(df['主要用途_from_info'].value_counts())
df['主要用途'] = df['主要用途_from_info']

(2415332, 9)
set()
(962995, 9)
(894637, 9)
主要用途
住家用     375453
商業用      14053
辦公用       9319
住商用       5198
工業用       3302
其他         505
住工用         82
工商用         50
商辦用         14
住商辦用         2
集合住宅         1
農業用          1
Name: count, dtype: int64
主要用途_from_info
住家用      260751
集合住宅     110652
商業用       15458
辦公室        7323
住商用        5193
一般事務所      4698
工業用        2489
廠房          874
店鋪          346
國民住宅        112
住工用          84
Name: count, dtype: int64


In [29]:
df['移轉層次'].describe()

count   407980.00000
mean         6.95614
std          5.14132
min          1.00000
25%          3.00000
50%          5.00000
75%         10.00000
max         50.00000
Name: 移轉層次, dtype: float64

In [30]:
df['總樓層數'].describe()

count   407980.00000
mean        12.20819
std          6.82349
min          1.00000
25%          7.00000
50%         12.00000
75%         15.00000
max         85.00000
Name: 總樓層數, dtype: float64

In [31]:
print(df.shape)
df = df.query('移轉層次 >= 2 and 移轉層次 <= 46')
print(df.shape)
df = df.query('總樓層數 >= 2 and 總樓層數 <= 68')
print(df.shape)
df = df.query('車位個數 >= 0 and 車位個數 <= 3')
print(df.shape)

(407980, 55)
(380527, 55)
(380382, 55)
(379144, 55)


In [32]:
df['Date'] = df['Trade_Year'].apply(str) + '-' + df['Trade_Month'].apply(lambda x: str(x).zfill(2)) + '-' + '01'
df['Date'] = pd.to_datetime(df['Date'])

In [33]:
df['key'] = df['縣市'] + '_' + df['鄉鎮市區'] + '_' + df['路名'] + '_' + df['建物型態'] + '_' + df['主要建材']
df = df.sort_values(by=['key', 'Date']).reset_index(drop=True)

ma_infos = []
ma_infos.append(df.groupby('key').rolling('50D', on='Date')['單價'].mean().reset_index().rename(columns={'單價': '單價_同路_50MA'}).drop_duplicates(subset=['key', 'Date'], keep='first').reset_index(drop=True))
ma_infos.append(df.groupby('key').rolling('100D', on='Date')['單價'].mean().reset_index().rename(columns={'單價': '單價_同路_100MA'}).drop_duplicates(subset=['key', 'Date'], keep='first').reset_index(drop=True))
ma_infos.append(df.groupby('key').rolling('200D', on='Date')['單價'].mean().reset_index().rename(columns={'單價': '單價_同路_200MA'}).drop_duplicates(subset=['key', 'Date'], keep='first').reset_index(drop=True))

ma_infos.append(df.groupby('key').rolling('50D', on='Date')['單價'].std().reset_index().rename(columns={'單價': '單價_同路_50MA_STD'}).drop_duplicates(subset=['key', 'Date'], keep='first').reset_index(drop=True))
ma_infos.append(df.groupby('key').rolling('100D', on='Date')['單價'].std().reset_index().rename(columns={'單價': '單價_同路_100MA_STD'}).drop_duplicates(subset=['key', 'Date'], keep='first').reset_index(drop=True))
ma_infos.append(df.groupby('key').rolling('200D', on='Date')['單價'].std().reset_index().rename(columns={'單價': '單價_同路_200MA_STD'}).drop_duplicates(subset=['key', 'Date'], keep='first').reset_index(drop=True))

for ma_info in ma_infos:
    df = df.merge(ma_info, how='left', on=['key', 'Date'])
df.head()

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,縣市,source,車位面積,建物面積,土地面積,Trade_Year,Trade_Month,Trade_YearMonth,Build_Year,Build_Month,屋齡,total_floor,floor,路名,車位個數,單價,格局_房,格局_廳,格局_衛,格局_隔間,格局_管理,主要用途_from_info,Date,key,單價_同路_50MA,單價_同路_100MA,單價_同路_200MA,單價_同路_50MA_STD,單價_同路_100MA_STD,單價_同路_200MA_STD
0,中區,房地(土地+建物),三民路二段６８號９樓之１,10.38000,商,NaN,NaN,1090626,土地2建物1車位0,9,12,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,711209.00000,93.28000,3,1,2,有,無,900000,9648.00000,NaN,0.00000,0,NaN,RPROMLQJPHPFFAB58EA,-0.38930,1.70050,-1.50560,有,NaN,台中市,2020Q3,-0.70228,-0.60767,-0.63458,2020,6,202006,1982,12,37.50000,12,9,三民路二段,0,0.16634,3,1,2,True,False,住家用,2020-06-01,台中市_中區_三民路二段_住宅大樓(11層含以上有電梯)_鋼筋混凝土造,0.16634,0.16634,0.16634,NaN,NaN,NaN
1,中區,房地(土地+建物),三民路二段２９號３樓之３,3.69000,商,NaN,NaN,1090708,土地2建物1車位0,3,12,住宅大樓(11層含以上有電梯),住商用,鋼筋混凝土造,730426.00000,40.79000,1,1,1,有,有,1460000,35793.00000,NaN,0.00000,0,NaN,RPUPMLOLPHPFFAB88DA,-1.23854,-0.65162,-1.50560,有,NaN,台中市,2020Q3,-0.70228,-1.60368,-0.92732,2020,7,202007,1984,4,36.25000,12,3,三民路二段,0,0.61712,1,1,1,True,True,住商用,2020-07-01,台中市_中區_三民路二段_住宅大樓(11層含以上有電梯)_鋼筋混凝土造,0.39173,0.39173,0.39173,0.31875,0.31875,0.31875
2,中區,房地(土地+建物),三民路二段４６號八樓之一,22.50000,商,NaN,NaN,1100401,土地1建物3車位0,8,12,住宅大樓(11層含以上有電梯),辦公室,鋼筋混凝土造,730811.00000,439.09000,6,0,2,有,有,19000000,43271.00000,NaN,0.00000,0,NaN,RPTNMLKKOHGGFAB17DA,3.33582,4.13810,-1.50560,有,NaN,台中市,2021Q3,-0.70228,2.83156,-0.29216,2021,4,202104,1984,8,36.66667,12,8,三民路二段,0,0.74605,6,0,2,True,True,辦公室,2021-04-01,台中市_中區_三民路二段_住宅大樓(11層含以上有電梯)_鋼筋混凝土造,0.74605,0.74605,0.74605,NaN,NaN,NaN
3,中區,房地(土地+建物),台中市中區三民路二段３７號十一樓之１,6.05000,商,NaN,NaN,1100621,土地1建物1車位0,11,12,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,791116.00000,67.11000,2,2,2,有,無,3450000,51408.00000,NaN,0.00000,0,NaN,RPPOMLLJRHGGFAB17DA,-0.92655,-0.65162,0.56642,有,1704.00000,台中市,2021Q3,-0.70228,-1.05403,-0.80609,2021,6,202106,1990,11,30.58333,12,11,三民路二段,0,0.88634,2,2,2,True,False,住家用,2021-06-01,台中市_中區_三民路二段_住宅大樓(11層含以上有電梯)_鋼筋混凝土造,0.88634,0.81620,0.81620,NaN,0.09920,0.09920
4,中區,房地(土地+建物),台中市中區三民路二段２９號５樓之５,6.74000,商,NaN,NaN,1100702,土地2建物2車位0,5,12,住宅大樓(11層含以上有電梯),住商用,鋼筋混凝土造,730426.00000,80.14000,2,0,2,有,有,2850000,35563.00000,NaN,0.00000,0,NaN,RPOOMLTJPHGGFAB67DA,-0.76465,-0.65162,1.21238,有,931.00000,台中市,2021Q4,-0.70228,-0.82264,-0.77538,2021,7,202107,1984,4,37.25000,12,5,三民路二段,0,0.61316,2,0,2,True,True,住商用,2021-07-01,台中市_中區_三民路二段_住宅大樓(11層含以上有電梯)_鋼筋混凝土造,0.74975,0.74852,0.74852,0.19317,0.13661,0.13661


In [34]:
df['key'] = df['縣市'] + '_' + df['鄉鎮市區'] + '_' + df['建物型態'] + '_' + df['主要建材']
df = df.sort_values(by=['key', 'Date']).reset_index(drop=True)

ma_infos = []
ma_infos.append(df.groupby('key').rolling('50D', on='Date')['單價'].mean().reset_index().rename(columns={'單價': '單價_同鄉鎮市區_50MA'}).drop_duplicates(subset=['key', 'Date'], keep='first').reset_index(drop=True))
ma_infos.append(df.groupby('key').rolling('100D', on='Date')['單價'].mean().reset_index().rename(columns={'單價': '單價_同鄉鎮市區_100MA'}).drop_duplicates(subset=['key', 'Date'], keep='first').reset_index(drop=True))
ma_infos.append(df.groupby('key').rolling('200D', on='Date')['單價'].mean().reset_index().rename(columns={'單價': '單價_同鄉鎮市區_200MA'}).drop_duplicates(subset=['key', 'Date'], keep='first').reset_index(drop=True))

ma_infos.append(df.groupby('key').rolling('50D', on='Date')['單價'].std().reset_index().rename(columns={'單價': '單價_同鄉鎮市區_50MA_STD'}).drop_duplicates(subset=['key', 'Date'], keep='first').reset_index(drop=True))
ma_infos.append(df.groupby('key').rolling('100D', on='Date')['單價'].std().reset_index().rename(columns={'單價': '單價_同鄉鎮市區_100MA_STD'}).drop_duplicates(subset=['key', 'Date'], keep='first').reset_index(drop=True))
ma_infos.append(df.groupby('key').rolling('200D', on='Date')['單價'].std().reset_index().rename(columns={'單價': '單價_同鄉鎮市區_200MA_STD'}).drop_duplicates(subset=['key', 'Date'], keep='first').reset_index(drop=True))

for ma_info in ma_infos:
    df = df.merge(ma_info, how='left', on=['key', 'Date'])
df.head()

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,縣市,source,車位面積,建物面積,土地面積,Trade_Year,Trade_Month,Trade_YearMonth,Build_Year,Build_Month,屋齡,total_floor,floor,路名,車位個數,單價,格局_房,格局_廳,格局_衛,格局_隔間,格局_管理,主要用途_from_info,Date,key,單價_同路_50MA,單價_同路_100MA,單價_同路_200MA,單價_同路_50MA_STD,單價_同路_100MA_STD,單價_同路_200MA_STD,單價_同鄉鎮市區_50MA,單價_同鄉鎮市區_100MA,單價_同鄉鎮市區_200MA,單價_同鄉鎮市區_50MA_STD,單價_同鄉鎮市區_100MA_STD,單價_同鄉鎮市區_200MA_STD
0,中區,房地(土地+建物),台中市中區民權路１６４號１８樓之１５,1.93000,住,NaN,NaN,1081002,土地2建物1車位0,18,28,住宅大樓(11層含以上有電梯),住家用,見其他登記事項,0860131,37.39000,1,1,1,有,無,1520000,40653.00000,NaN,0.00000,0,NaN,RPVPMLSKJIOFFAB58CA,-1.51959,0.66089,-1.50560,NaN,NaN,台中市,2020Q1,-0.70228,-1.68649,-1.04698,2019,10,201910,1997,1,22.75000,28,18,民權路,0,0.70091,1,1,1,True,False,住家用,2019-10-01,台中市_中區_住宅大樓(11層含以上有電梯)_見其他登記事項,0.70091,0.70091,0.70091,NaN,NaN,NaN,0.70091,0.70091,0.70091,NaN,NaN,NaN
1,中區,房地(土地+建物),台中市中區民權路１６４號１７樓之３５,1.93000,商,NaN,NaN,1081030,土地2建物1車位0,17,28,住宅大樓(11層含以上有電梯),住家用,見其他登記事項,0860131,37.55000,1,0,1,有,有,1937500,51598.00000,NaN,0.00000,0,NaN,RPSOMLPKJIOFFAB87CA,-1.54485,0.62246,-1.50560,NaN,NaN,台中市,2020Q1,-0.70228,-1.68251,-1.04698,2019,10,201910,1997,1,22.75000,28,17,民權路,0,0.88962,1,0,1,True,True,住家用,2019-10-01,台中市_中區_住宅大樓(11層含以上有電梯)_見其他登記事項,0.70091,0.70091,0.70091,NaN,NaN,NaN,0.70091,0.70091,0.70091,NaN,NaN,NaN
2,中區,房地(土地+建物),台中市中區民權路１６４號１７樓之３６,1.93000,商,NaN,NaN,1081105,土地2建物1車位0,17,28,住宅大樓(11層含以上有電梯),住家用,見其他登記事項,0860131,37.55000,1,0,1,有,有,1750000,46605.00000,NaN,0.00000,0,NaN,RPRNMLSJKIOFFAB97CA,-1.54485,0.62246,-1.50560,NaN,NaN,台中市,2020Q1,-0.70228,-1.68251,-1.04698,2019,11,201911,1997,1,22.83333,28,17,民權路,0,0.80353,1,0,1,True,True,住家用,2019-11-01,台中市_中區_住宅大樓(11層含以上有電梯)_見其他登記事項,0.79802,0.79802,0.79802,0.09447,0.09447,0.09447,0.79802,0.79802,0.79802,0.09447,0.09447,0.09447
3,中區,房地(土地+建物),台中市中區民權路１６４號２６樓之１４及１５,3.86000,商,NaN,NaN,1081110,土地2建物2車位0,26,28,住宅大樓(11層含以上有電梯),住家用,見其他登記事項,0860131,74.79000,1,1,1,有,有,3490000,46664.00000,NaN,0.00000,0,二戶打通，合併計價,RPTNMLMLJHPFFAB67CA,-0.89763,1.20455,-1.50560,NaN,NaN,台中市,2020Q1,-0.70228,-0.91518,-0.91748,2019,11,201911,1997,1,22.83333,28,26,民權路,0,0.80455,1,1,1,True,True,住家用,2019-11-01,台中市_中區_住宅大樓(11層含以上有電梯)_見其他登記事項,0.79802,0.79802,0.79802,0.09447,0.09447,0.09447,0.79802,0.79802,0.79802,0.09447,0.09447,0.09447
4,中區,房地(土地+建物),台中市中區民權路１６４號２３樓之３８,1.93000,商,NaN,NaN,1081219,土地2建物1車位0,23,28,住宅大樓(11層含以上有電梯),住家用,見其他登記事項,0860131,37.55000,1,0,1,有,有,1800000,47936.00000,NaN,0.00000,0,NaN,RPRRMLKKJHPFFAB57CA,-1.54485,0.62246,-1.50560,NaN,NaN,台中市,2020Q1,-0.70228,-1.68251,-1.04698,2019,12,201912,1997,1,22.91667,28,23,民權路,0,0.82648,1,0,1,True,True,住家用,2019-12-01,台中市_中區_住宅大樓(11層含以上有電梯)_見其他登記事項,0.81152,0.80502,0.80502,0.01297,0.06793,0.06793,0.81152,0.80502,0.80502,0.01297,0.06793,0.06793


In [35]:
use_sources = ['2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4', '2023Q1', '2023Q2', '2023Q3']

print(df.shape)
df = df[df['source'].isin(use_sources)].copy().reset_index(drop=True)
print(df.shape)
df = df.query('Trade_Year >= 2021 and Trade_Year <= 2022')
print(df.shape)

(379144, 69)
(268002, 69)
(208127, 69)


In [36]:
price_cols = [c for c in df.columns if '單價' in c]
use_cols = ['縣市', '鄉鎮市區', '路名', '建物型態', '主要建材', '總樓層數', '移轉層次', 
            '格局_房', '格局_廳', '格局_衛', '格局_隔間', '格局_管理',
            '主要用途', '車位個數', '屋齡', '土地面積', '建物面積', '車位面積', '主建物面積', '陽台面積', '附屬建物面積'] + price_cols

df[use_cols].to_csv('../外部資料集/實價登錄/external_gov_data.csv', index=False)
df[use_cols].head(10)

,縣市,鄉鎮市區,路名,建物型態,主要建材,總樓層數,移轉層次,格局_房,格局_廳,格局_衛,格局_隔間,格局_管理,主要用途,車位個數,屋齡,土地面積,建物面積,車位面積,主建物面積,陽台面積,附屬建物面積,單價元平方公尺,單價,單價_同路_50MA,單價_同路_100MA,單價_同路_200MA,單價_同路_50MA_STD,單價_同路_100MA_STD,單價_同路_200MA_STD,單價_同鄉鎮市區_50MA,單價_同鄉鎮市區_100MA,單價_同鄉鎮市區_200MA,單價_同鄉鎮市區_50MA_STD,單價_同鄉鎮市區_100MA_STD,單價_同鄉鎮市區_200MA_STD
2,台中市,中區,自由路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,11,6,1,0,1,True,True,住家用,0,43.50000,-0.71756,-1.35438,-0.70228,-0.72380,-1.50560,-0.65162,30239.00000,0.52136,0.52136,0.52136,0.60444,NaN,NaN,0.11749,0.66132,0.66132,0.70570,0.19793,0.19793,0.13545
3,台中市,中區,民權路,住宅大樓(11層含以上有電梯),鋼筋混凝土造,12,6,1,0,1,True,True,集合住宅,0,24.33333,-0.96211,-1.80290,-0.70228,-1.64130,-1.50560,0.70841,45648.00000,0.78703,0.78703,0.78703,0.78703,NaN,NaN,NaN,0.65420,0.70322,0.77362,0.18786,0.15766,0.14802
4,台中市,中區,三民路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,12,8,6,0,2,True,True,辦公室,0,36.66667,-0.29216,2.83156,-0.70228,3.33582,-1.50560,4.13810,43271.00000,0.74605,0.74605,0.74605,0.74605,NaN,NaN,NaN,0.74605,0.68482,0.71393,NaN,0.14303,0.13050
5,台中市,中區,建國路,住宅大樓(11層含以上有電梯),鋼筋混凝土造,12,6,0,0,0,False,True,商業用,0,39.91667,-0.54355,0.37649,-0.70228,0.87915,-1.50560,-0.65162,42126.00000,0.72631,0.72631,0.72631,0.72631,NaN,NaN,NaN,0.74605,0.68482,0.71393,NaN,0.14303,0.13050
6,台中市,中區,三民路２段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,12,10,0,0,2,True,True,住商用,0,36.75000,0.06366,2.83156,-0.70228,3.33582,2.75868,-0.65162,43955.00000,0.75784,0.75784,0.75784,0.75784,NaN,NaN,NaN,0.74340,0.75431,0.72331,0.01593,0.02540,0.10261
7,台中市,中區,中華路一段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,12,5,1,1,1,True,False,住家用,0,39.41667,-1.00809,-1.94693,-0.70228,-1.61684,-1.50560,0.60721,54230.00000,0.93500,0.93500,0.93500,0.93500,NaN,NaN,NaN,0.74340,0.75431,0.72331,0.01593,0.02540,0.10261
8,台中市,中區,民權路,住宅大樓(11層含以上有電梯),鋼筋混凝土造,12,12,2,0,2,True,True,集合住宅,0,24.58333,-0.81625,-1.15997,-0.70228,-1.12846,0.12830,-0.65162,73135.00000,1.26095,1.26095,1.02399,1.02399,NaN,0.33511,0.33511,0.74340,0.75431,0.72331,0.01593,0.02540,0.10261
9,台中市,中區,三民路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,12,11,2,2,2,True,False,住家用,0,30.58333,-0.80609,-1.05403,-0.70228,-0.92655,0.56642,-0.65162,51408.00000,0.88634,0.88634,0.81620,0.81620,NaN,0.09920,0.09920,0.96003,0.88542,0.82469,0.21408,0.20224,0.20019
10,台中市,中區,大誠街,住宅大樓(11層含以上有電梯),鋼筋混凝土造,12,12,1,1,1,True,True,住家用,0,26.58333,-0.93026,-1.66225,-0.70228,-1.39886,-1.50560,-0.13148,61897.00000,1.06719,1.06719,1.06719,1.06719,NaN,NaN,NaN,0.96003,0.88542,0.82469,0.21408,0.20224,0.20019
11,台中市,中區,三民路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,12,5,2,0,2,True,True,住商用,0,37.25000,-0.77538,-0.82264,-0.70228,-0.76465,1.21238,-0.65162,35563.00000,0.61316,0.74975,0.74852,0.74852,0.19317,0.13661,0.13661,0.85556,0.87411,0.83012,0.22858,0.21066,0.21687
